In [ ]:
import numpy as np
import processingIO as pio
import matplotlib.pyplot as plt
import pandas as pd

from pathlib import Path
from dataUtilities import filterData
from forceClass import Forces

from scipy import stats
from scipy.optimize import curve_fit
from scipy.stats.distributions import  t

from mpl_toolkits.axes_grid1.inset_locator import (inset_axes, InsetPosition,
                                                  mark_inset)

In [ ]:
def power_law(xData, a, b, c):
        return a + (b * xData) + (c * xData**2)

def fit_data(xData, yData):

    popt, pcov = curve_fit(power_law, xData, yData, p0=[0.5, 0.5, 0.5])

    residuals = yData - power_law(xData, *popt)

    ss_res = np.sum(residuals**2)
    ss_total = np.sum((yData-np.mean(yData))**2)

    r_squared = 1 - (ss_res/ss_total)

    return popt, r_squared

In [ ]:
data_location = r'/location/of/data'

force_paths = pio.get_files(data_location, 'force.dat')
force_paths.sort()
solver_paths = pio.get_files(data_location, 'log.solver')
solver_paths.sort()

tmp = []

constantDict = {'parentCase':[], 'specificCase':[], 'totalTime':[]}

for solverFile in solver_paths:
    constantDict['parentCase'].append(solverFile.parts[-4])
    constantDict['specificCase'].append(solverFile.parts[-3])
    with open(solverFile, 'r') as f:
        for line in f:
            if 'ExecutionTime' in line:
                tmp.append(line.split()[2])

    constantDict['totalTime'].append(tmp[-1]) 

cycles = 1.0
total_cycles = 2.5

forces = [Forces(filepath, cycles, total_cycles, True, True) for filepath in force_paths]

In [ ]:
density = 998.2
fudgeFactor = 1.0
alpha_max = [5, 10, 15, 20, 25, 30]
average_fx = []
case = []
number_subCases = 7

for force in forces:
    average_fx.append(force.averageFilteredForces['total']['x']/density)
    case.append(force.specific_case)

fig1, ax1 = plt.subplots(figsize=(12,10))
for force in forces:
    ax1.plot(force.filteredForces['time'], force.filteredForces['total']['x'], label=force.specific_case)

#ax2 = plt.axes([0, 0, 1, 1])
#ip = InsetPosition(ax1, [0.2,0.4,0.55,0.55])
#ax2.set_axes_locator(ip)
#mark_inset(ax1, ax2, loc1=2, loc2=4, fc="none", ec='0.5')

#ax1.legend()
ax1.set_ylim([-3000, 3000])

In [ ]:
case1 = []
case2 = []
number_cases = 7
U = 1.0
chord = 1.0
density = 1025
fudgeFactor = 1.0

a_st = [0.10, 0.20, 0.25, 0.30, 0.35, 0.40, 0.50]

anderson_data = r'/location/of/Anderson/data'
andersonDF = pd.read_excel(anderson_data, sheet_name='alpha15')

for i in range(number_cases):
    case1.append(forces[i].averageFilteredForces['total']['x']/(0.5*density*U**2*chord))
    case2.append(forces[i+number_cases].averageFilteredForces['total']['x']/(0.5*density*U**2*chord))
    #case3.append(forces[i+18].averageFilteredForces['total']['x']/density)
    #case4.append(forces[i+27].averageFilteredForces['total']['x']/density)

case1 = np.asarray(case1)
case2 = np.asarray(case2)

fig1, ax1 = plt.subplots(figsize=(12,10))
ax1.plot(andersonDF['Strouhal'], andersonDF['Ct'], 'o-', label='Anderson Data')
ax1.plot(a_st, case1*fudgeFactor, 'o-', label='Case 1')
ax1.plot(a_st, case2*fudgeFactor, 'o-', label='Case 2')
#ax1.plot(e_st, case3*fudgeFactor, 'o-', label='Case 3')
#ax1.plot(e_st, case4*fudgeFactor, 'o-', label='Case 4')

#ax2 = plt.axes([0, 0, 1, 1])
#ip = InsetPosition(ax1, [0.2,0.2,0.55,0.55])
#ax2.set_axes_locator(ip)
#mark_inset(ax1, ax2, loc1=2, loc2=4, fc="none", ec='0.5')

#startIndex = 0
#endIndex = 4

#ax2.plot(e_st[startIndex:endIndex], e_force[startIndex:endIndex], 'o-', label='Ebrahim Data')
#ax2.plot(e_st[startIndex:endIndex], case1[startIndex:endIndex]*fudgeFactor, 'o-', label='Case 1')
#ax2.plot(e_st[startIndex:endIndex], case2[startIndex:endIndex]*fudgeFactor, 'o-', label='Case 2')
#ax2.plot(e_st[startIndex:endIndex], case3[startIndex:endIndex]*fudgeFactor, 'o-', label='Case 3')
#ax2.plot(e_st[startIndex:endIndex], case4[startIndex:endIndex]*fudgeFactor, 'o-', label='Case 4')
#ax2.set_ylim([-4, 1.5])

ax1.legend()
#ax1.set_ylim([0.5, -40])

In [ ]:
constantDict